In [1]:
%%bash
opam install frama-c
why3 config detect

The following actions will be performed:
  ∗ install   fpath                0.7.3    [required by bos]
  ∗ install   conf-gtk3            18       [required by lablgtk3]
  ∗ install   conf-cairo           1        [required by cairo2]
  ∗ install   unionFind            20220122 [required by frama-c]
  ∗ install   ordering             3.13.0   [required by dyn, stdune]
  ∗ install   ppx_deriving_yojson  3.7.0    [required by frama-c]
  ∗ install   logs                 0.7.0    [required by ocplib-simplex, bos]
  ∗ install   conf-zlib            1        [required by camlzip]
  ∗ install   psmt2-frontend       0.4.0    [required by alt-ergo-parsers]
  ∗ install   ppx_blob             0.7.2    [required by alt-ergo-lib]
  ∗ install   dolmen               0.9      [required by alt-ergo-lib]
  ∗ install   ocamlgraph           2.1.0    [required by frama-c]
  ∗ install   pp                   1.2.0    [required by dune-private-libs]
  ∗ install   pp_loc               2.1.0    [required by dol

I consider Frama-C to be the big dog in the room when it comes to C verification.

mlcfg + pcode?

https://frama-c.com/2022/11/16/supertest-solidsands.html This was already the case with NIST’s Juliet C/C++ testsuite, [J. Moerman’s 2018 bachelor’s thesis test suite (JM2018TS)(https://github.com/JMoerman/JM2018TS), TrailOfBits’ version of the DARPA Challenge Binaries, 

https://en.wikipedia.org/wiki/Frama-C

- eva evolved value analysis https://frama-c.com/fc-plugins/eva.html
- wp
- jessie - old. kind of replaced by wp?
- e-acsl - adds runtime assertions
- Pathcraewler - https://frama-c.com/fc-plugins/pathcrawler.html generate unit tests. proprietary

- rte - runtime error detection. preprocessor to wp or e-acsl
- ltl properties https://frama-c.com/fc-plugins/aorai.html
- https://frama-c.com/fc-plugins/metrics-calculation.html metrics. cyclomatic, yuck. but eva coverage ok
- https://frama-c.com/fc-plugins/instantiate.html specialize function signatures?
- constant folding + useless code elimination. A frameowrk for partial eval?

eva is probably the most scalable thing?

invariant inference?


In [3]:
%%file /tmp/frama1.c

#include <stdio.h>
int main() {
    int a = 1;
    int b = 2;
    int c = a + b;
    printf("%d\n", c);
    return 0;
}

Writing /tmp/frama1.c


In [5]:
! frama-c -eva /tmp/frama1.c

[kernel] Parsing /tmp/frama1.c (with preprocessing)
[eva] Analyzing a complete application starting at main
[eva:initial-state] Values of globals at initialization
  
[eva] using specification for function printf_va_1
[eva] ====== VALUES COMPUTED ======
[eva:final-states] Values at end of function main:
  a ∈ {1}
  b ∈ {2}
  c ∈ {3}
  __retres ∈ {0}
  S___fc_stdout[0..1] ∈ [--..--]
[eva:summary] ====== ANALYSIS SUMMARY ======
  ----------------------------------------------------------------------------
  1 function analyzed (out of 1): 100% coverage.
  In this function, 6 statements reached (out of 6): 100% coverage.
  ----------------------------------------------------------------------------
  No errors or warnings raised during the analysis.
  ----------------------------------------------------------------------------
  0 alarms generated by the analysis.
  ----------------------------------------------------------------------------
  Evaluation of the logical properties reached 

In [9]:
%%file /tmp/frama2.c
// from https://frama-c.com/fc-plugins/eva.html
int abs(int x) {
  if (x < 0) return -x;
  else return x;
}

Overwriting /tmp/frama2.c


In [10]:
! frama-c -eva /tmp/frama2.c -main abs 

[kernel] Parsing /tmp/frama2.c (with preprocessing)
[eva] Analyzing a complete application starting at abs
[eva:initial-state] Values of globals at initialization
  
[eva:alarm] /tmp/frama2.c:3: Warning: signed overflow. assert -x ≤ 2147483647;
[eva] ====== VALUES COMPUTED ======
[eva:final-states] Values at end of function abs:
  __retres ∈ [0..2147483647]
[eva:summary] ====== ANALYSIS SUMMARY ======
  ----------------------------------------------------------------------------
  1 function analyzed (out of 1): 100% coverage.
  In this function, 8 statements reached (out of 8): 100% coverage.
  ----------------------------------------------------------------------------
  No errors or warnings raised during the analysis.
  ----------------------------------------------------------------------------
  1 alarm generated by the analysis:
       1 integer overflow
  ----------------------------------------------------------------------------
  No logical properties have been reached by th

In [11]:
%%file /tmp/ex1.c
#include <assert.h>
int main(){
    int x;
    if (x <= 42){
            assert(x != 12345);
    }
}

Overwriting /tmp/ex1.c


In [12]:
! frama-c -eva /tmp/ex1.c

[kernel] Parsing /tmp/ex1.c (with preprocessing)
[eva] Analyzing a complete application starting at main
[eva:initial-state] Values of globals at initialization
  
[eva:alarm] /tmp/ex1.c:4: Warning: 
  accessing uninitialized left-value. assert \initialized(&x);
[eva] /tmp/ex1.c:4: assertion 'Eva,initialization' got final status invalid.
[eva] ====== VALUES COMPUTED ======
[eva:final-states] Values at end of function main:
  NON TERMINATING FUNCTION
[eva:summary] ====== ANALYSIS SUMMARY ======
  ----------------------------------------------------------------------------
  1 function analyzed (out of 1): 100% coverage.
  In this function, 1 statements reached (out of 4): 25% coverage.
  ----------------------------------------------------------------------------
  No errors or warnings raised during the analysis.
  ----------------------------------------------------------------------------
  1 alarm generated by the analysis:
       1 access to uninitialized left-values
  1 of them is

In [4]:
! frama-c -wp -wp-rte /tmp/frama1.c

[kernel] Parsing /tmp/frama1.c (with preprocessing)
[rte:annot] annotating function main
[kernel:annot:missing-spec] FRAMAC_SHARE/libc/stdio.h:211: Warning: 
  Neither code nor explicit exits and terminates for function printf_va_1,
   generating default clauses. See -generated-spec-* options for more info
[wp] 5 goals scheduledsage
[wp] [Timeout] typed_main_call_printf_va_1_requires (Qed 6ms) (Alt-Ergo)
[wp] Proved goals:    4 / 5
  Qed:             4
  Timeout:         1
[wp:pedantic-assigns] /tmp/frama1.c:3: Warning: 
  No 'assigns' specification for function 'main'.
  Callers assumptions might be imprecise.
[wp] FRAMAC_SHARE/libc/stdio.h:211: Warning: 
  Memory model hypotheses for function 'printf_va_1':
  /*@
     behavior wp_typed:
       requires \separated(&__fc_stdout, &__fc_stdout->__fc_FILE_data);
       requires \separated(format + (..), &__fc_stdout);
     */
  int printf_va_1(char const * restrict format, int param0);


eva is a value analysis? 

In [2]:
! frama-c -plugins

Alias Analysis      Lightweight May-Alias Analysis (experimental) (-alias-h)
Aorai               verification of behavioral properties (experimental)
                    (-aorai-h)
Callgraph           automatically compute the callgraph of the program. Using
                    Eva might improve the precision of this plug-in (-cg-h)
Dive                An interactive imprecision graph generator for Eva.
                    (-dive-h)
Dominators          Compute postdominators of statements (-dominators-h)
E-ACSL              Executable ANSI/ISO C Specification Language --- runtime
                    assertion checker generator (-e-acsl-h)
Eva                 automatically computes variation domains for the
                    variables of the program (-eva-h)
From analysis       functional dependencies (-from-h)
Impact              impact analysis (-impact-h)
Inout               operational, imperative and all kinds of inputs/outputs
                    (-inout-h)
Instantiate         O